<a href="https://colab.research.google.com/github/JapiKredi/RAG_assignment_Research_papers/blob/main/RAG_assignment_Research_papers_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment "Course 3 Mr.HelpMate"
### Learner: Jasper Bongers
#### Option 1: Build Your Own Project (BYOP)

- I a building a generative AI system that is reading 5 popular Research Papers in Natural Language Understanding.
- I am using 3 layers: the emebdding layer, the search layer, and the generation layer.
- My goal in this session is to build a robust search system that can answer user queries effectively by experimenting with the various blocks in the system, such as chunking strategies, embedding models, re-rankers, generation prompt.

# <font color = green> Solution 1


## 1. <font color = red> Install and Import the Required Libraries

Please run the following code on a GPU, as running it on a CPU can create a crash

In [1]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Import all the required Libraries
import openai
import pypdf
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import chromadb

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from sentence_transformers import CrossEncoder, util

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [ ]:
!pip show langchain

## 2. <font color = red> Read, Process, and Chunk the PDF Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Set the API key
filepath = "/content/drive/My Drive/GenerativeAI/MateAI/"

with open(filepath + "Jasper_OpenAI_API_Key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = openai.api_key

## Load multiple and process documents

In [ ]:
# Load and process the text files
loader = DirectoryLoader("/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/", glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [ ]:
len(documents)

## Splitting, Chunking of text

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

In [ ]:
texts[10]

## Embedding the text via regular HuggingFace process

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


### Creating Chroma Vector Database

In [ ]:
# Define the path where chroma collections will be stored
chroma_data_path = '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = chroma_data_path

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## 4. <font color = red> Semantic Search

In this section, we will perform a semantic search of a query in the collections embeddings to get several top semantically similar results.

In [ ]:
## Make a retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

In [ ]:
# Get a relevant query
query = "What is Flash attention?"

In [ ]:
docs = retriever.get_relevant_documents("What is Flash attention?")

In [ ]:
retriever.search_type

## Make a chain

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

## 6. Retrieval Augmented Generation

Now that we have the final top search results, we can pass it to an GPT 3.5 along with the user query and a well-engineered prompt, to generate a direct answer to the query along with citations, rather than returning whole pages/chunks.

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# Query 1: What is Flash attention?
query = "What is Flash attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
# Query 2: What is self attention?
query = "What is self-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
# Query 3: What is multi head attention?
query = "What is multi head attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
# Query 4: What does IO-aware mean?
query = "What does IO-aware mean?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
# Query 5: What is tiling in flash-attention?
query = "What is tiling in flash-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
# Query 6: What tools can be used with toolformer?
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
# Query 7: What are the best retrieval augmentations for LLMs?
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
user_query = input("What is your query about the AI research papers? ")

In [ ]:
# Querry 8: What are the best retrieval augmentations for LLMs?
llm_response = qa_chain(user_query)
process_llm_response(llm_response)

## Deleting the Chroma Vector Database

In [ ]:
# Define the path where chroma collections will be stored
# chroma_data_path = '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [ ]:
# !zip -r db.zip ./db
!zip -r ChromaDB_Data.zip '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

## Starting again loading the db

restart the runtime

In [ ]:
!unzip db.zip
!unzip '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data.zip'